In [1]:
import QuantLib as ql

// Load call schedules
for (Size i=0; i<LENGTH(callLength); i++) {
    callability.push_back(
           ext::shared_ptr<Callability>(
               new SoftCallability(Callability::Price(
                                           callPrices[i],
                                           Callability::Price::Clean),
                                   schedule.date(callLength[i]),
                                   1.20)));
}

for (Size j=0; j<LENGTH(putLength); j++) {
    callability.push_back(
           ext::make_shared<Callability>(
                   Callability::Price(
                                           putPrices[j],
                                           Callability::Price::Clean),
                                   Callability::Put,
                                   schedule.date(putLength[j])));
}

// Assume dividends are paid every 6 months.
for (Date d = today + 6*Months; d < exerciseDate; d += 6*Months) {
    dividends.push_back(
              ext::shared_ptr<Dividend>(new FixedDividend(1.0, d)));
}

DayCounter dayCounter = Actual365Fixed();
Time maturity = dayCounter.yearFraction(settlementDate,
                                        exerciseDate);

In [5]:
Type = ql.Option.Put
underlying = 36.0
spreadRate = 0.005

dividendYield = 0.02
riskFreeRate = 0.06
volatility = 0.20

settlementDays = 3
length = 5
redemption = 100.0
conversionRatio = redemption/underlying #ATM

In [7]:
calendar = ql.TARGET()
today = calendar.adjust(ql.Date.todaysDate())

ql.Settings.instance().evaluationDate = today
settlementDate = calendar.advance(today, settlementDays, ql.Days)
exerciseDate = calendar.advance(settlementDate, length, ql.Years)
issueDate = calendar.advance(exerciseDate, -length, ql.Years)

In [8]:
convention = ql.ModifiedFollowing

frequency = ql.Annual

schedule = ql.Schedule(issueDate, exerciseDate,
                  ql.Period(frequency), calendar,
                  convention, convention,
                  ql.DateGeneration.Backward, False)

In [18]:

d = today + ql.Period(6,ql.Months)
dividend_dates=[d]
while d < exerciseDate:
    d += ql.Period(6, ql.Months)
    dividend_dates+=[d]

dividends = ql.DividendSchedule([ql.FixedDividend(1.0,d) for d in dividend_dates])
callability = ql.CallabilitySchedule()

coupons = [1, 0.05]

bondDayCount = ql.Thirty360()

In [10]:
callLength = [ 2, 4 ]  # Call dates, years 2, 4.
putLength = [ 3 ] # Put dates year 3

callPrices = [ 101.5, 100.85 ]
putPrices= [ 105.0 ]

In [11]:
callability.appe

<QuantLib.QuantLib.CallabilitySchedule; proxy of <Swig Object of type 'std::vector< boost::shared_ptr< Callability >,std::allocator< boost::shared_ptr< Callability > > > *' at 0x000001206B7C3AB0> >

In [23]:
exercise = ql.EuropeanExercise(exerciseDate)
creditSpread = ql.QuoteHandle(ql.SimpleQuote(spreadRate))

In [24]:
europeanBond = ql.ConvertibleFixedCouponBond (
                    exercise, conversionRatio, dividends, callability,
                    creditSpread, issueDate, settlementDays,
                    coupons, bondDayCount, schedule, redemption)

In [28]:
underlyingH = ql.QuoteHandle(ql.SimpleQuote(underlying))

In [38]:
dayCounter = ql.Actual365Fixed()
flatTermStructure = ql.YieldTermStructureHandle(ql.FlatForward(settlementDate, riskFreeRate, dayCounter))
flatDividendTS = ql.YieldTermStructureHandle(ql.FlatForward(settlementDate, dividendYield, dayCounter))
flatVolTS = ql.BlackVolTermStructureHandle(ql.BlackConstantVol(settlementDate, calendar, volatility, dayCounter))

In [39]:
stochasticProcess = ql.BlackScholesMertonProcess(underlyingH,
                                                flatDividendTS,
                                                flatTermStructure,
                                                flatVolTS)

In [41]:
timeSteps = 801

In [42]:
engine = ql.BinomialConvertibleEngine(stochasticProcess, "jarrowrudd",
                                                            timeSteps)

In [43]:
europeanBond.setPricingEngine(engine)

In [44]:
europeanBond.NPV()

191.50785052843446